In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from functools import reduce
from datetime import datetime

In [2]:
data_atual = datetime.now().strftime("%Y-%m-%d")
print("Data Atual:", data_atual)

Data Atual: 2024-06-03


In [3]:
spark = SparkSession.builder.appName("App_Handler_Join").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/03 07:38:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_clientes = spark.read.option("delimiter", ",").option("header", True).csv("clientes.csv")\
.withColumn("id_cliente", F.col("id_cliente").cast("integer"))\
.withColumn("nome", F.col("nome").cast("string"))\
.withColumn("genero", F.col("genero").cast("string"))\
.withColumn("data_nascimento", F.col("data_nascimento").cast("date"))\
.withColumn("idade", (F.months_between(F.current_date(), F.col("data_nascimento")) / 12).cast("integer"))\

df_clientes.show(truncate=False)

+----------+--------+------+---------------+-----+
|id_cliente|nome    |genero|data_nascimento|idade|
+----------+--------+------+---------------+-----+
|1         |Anderson|M     |1980-11-06     |43   |
|2         |Fernanda|F     |1979-04-12     |45   |
|3         |Gustavo |M     |2019-02-20     |5    |
|4         |João    |M     |2013-03-01     |11   |
+----------+--------+------+---------------+-----+



In [5]:
df_produtos = spark.read.option("delimiter", ",").option("header", True).csv("produtos.csv")\
.withColumn("id_produto", F.col("id_produto").cast("integer"))\
.withColumn("produto", F.col("produto").cast("string"))\
.withColumn("preco", F.col("preco").cast("float"))\

df_produtos.show(truncate=False)

+----------+-----------------------+-----+
|id_produto|produto                |preco|
+----------+-----------------------+-----+
|1         |Suco de Laranja 500ml  |4.99 |
|2         |Pão de Queijo          |4.99 |
|3         |Batata Frita           |15.99|
|4         |Cerveja Heineken 600ml |13.99|
|5         |Garrafa de Vinho Merlot|89.99|
|6         |Garrafa de Vinho Verde |89.99|
|7         |Garrafa de Vinho Rosê  |89.99|
|8         |Filé com Fritas        |69.99|
|9         |Prato Kids             |19.99|
+----------+-----------------------+-----+



In [6]:
df_pedidos = spark.read.option("delimiter", ",").option("header", True).csv("pedidos.csv")\
.withColumn("id_pedido", F.col("id_pedido").cast("integer"))\
.withColumn("id_cliente", F.col("id_cliente").cast("integer"))\
.withColumn("id_produto", F.col("id_produto").cast("integer"))\

df_pedidos.show(truncate=False)

+---------+----------+----------+
|id_pedido|id_cliente|id_produto|
+---------+----------+----------+
|1        |1         |4         |
|2        |2         |5         |
|3        |4         |1         |
|4        |4         |3         |
|5        |4         |8         |
|6        |1         |4         |
|7        |3         |2         |
|8        |2         |5         |
|9        |2         |6         |
|10       |3         |1         |
|11       |4         |2         |
|12       |3         |8         |
|13       |3         |1         |
+---------+----------+----------+



In [7]:
df_ticket = df_pedidos.alias("t1")\
.join(df_clientes.alias("t2"),(F.col("t1.id_cliente") == F.col("t2.id_cliente")), "left")\
.join(df_produtos.alias("t3"),(F.col("t1.id_produto") == F.col("t3.id_produto")), "left")\
.select("t1.id_pedido", "t2.nome", "t3.produto", "t3.preco")\

df_ticket.show(truncate=False)

+---------+--------+-----------------------+-----+
|id_pedido|nome    |produto                |preco|
+---------+--------+-----------------------+-----+
|1        |Anderson|Cerveja Heineken 600ml |13.99|
|2        |Fernanda|Garrafa de Vinho Merlot|89.99|
|3        |João    |Suco de Laranja 500ml  |4.99 |
|4        |João    |Batata Frita           |15.99|
|5        |João    |Filé com Fritas        |69.99|
|6        |Anderson|Cerveja Heineken 600ml |13.99|
|7        |Gustavo |Pão de Queijo          |4.99 |
|8        |Fernanda|Garrafa de Vinho Merlot|89.99|
|9        |Fernanda|Garrafa de Vinho Verde |89.99|
|10       |Gustavo |Suco de Laranja 500ml  |4.99 |
|11       |João    |Pão de Queijo          |4.99 |
|12       |Gustavo |Filé com Fritas        |69.99|
|13       |Gustavo |Suco de Laranja 500ml  |4.99 |
+---------+--------+-----------------------+-----+



24/06/03 08:01:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 127907 ms exceeds timeout 120000 ms
24/06/03 08:01:44 WARN SparkContext: Killing executors is not supported by current scheduler.
24/06/03 08:01:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$